In [31]:
import sys
import os
sys.path.append("../")

In [32]:
import os
import cv2
import numpy as np
import albumentations as alb
from scripts.preprocessing import *
from helpers.model_handler import *
import matplotlib.pyplot as plt
import random

WARNING DOES NOT ADD ORIGINAL DATASET. SHOULD ALREADY BE CONTAINED IN DIR

In [33]:
IMAGE_DIR = "../dataset/training/images/"
GT_DIR = "../dataset/training/groundtruth/"
IMAGE_DIR_AUGMENTED = "../dataset/training/augmented_images_big/"
GT_DIR_AUGMENTED = "../dataset/training/augmented_groundtruth_big/"
FILES = sorted(os.listdir(IMAGE_DIR))
N_IMGS = 100

In [34]:
def save_augmented(images, masks, probas1=[0.8], probas2=[0.5], probas3=[0.5]):
    assert len(images) == len(masks)
    
    if not os.path.isdir(IMAGE_DIR_AUGMENTED):
        os.mkdir(IMAGE_DIR_AUGMENTED)
    if not os.path.isdir(GT_DIR_AUGMENTED):
        os.mkdir(GT_DIR_AUGMENTED)
        
    

    for proba1, proba2, proba3 in zip(probas1, probas2, probas3):
        existing = len(os.listdir(IMAGE_DIR_AUGMENTED))
        for i in range(len(images)): 

            image = img_float_to_uint8(images[i])
            mask = img_float_to_uint8(masks[i])
            aug = get_transform(proba1, proba2)(image=image, mask=mask)
            img_aug = aug['image']
            msk_aug = aug['mask']

            if random.random() < proba3:
                img_aug, msk_aug = rotate_inner(img_aug, msk_aug)

            img_name = f"satImage_{str(existing+i).zfill(3)}.png"
            msk_name = f"satImage_{str(existing+i).zfill(3)}.png"

            good = cv2.imwrite(os.path.join(IMAGE_DIR_AUGMENTED, img_name), img_aug)
            good2 = cv2.imwrite(os.path.join(GT_DIR_AUGMENTED, msk_name), msk_aug)
            if not good:
                print("Error saving image: ", img_name)
            if not good2:
                print("Error saving mask: ", msk_name)


# Pixel_transforms (label stay unchanged): 

# AdvancedBlur
# Blur
# CLAHE
# ChannelDropout
# ChannelShuffle
# ColorJitter
# Defocus
# Downscale
# Emboss
# Equalize
# FDA
# FancyPCA
# FromFloat
# GaussNoise
# GaussianBlur
# GlassBlur
# HistogramMatching
# HueSaturationValue
# ISONoise
# ImageCompression
# InvertImg
# MedianBlur
# MotionBlur
# MultiplicativeNoise
# Normalize
# PixelDistributionAdaptation
# Posterize
# RGBShift
# RandomBrightnessContrast
# RandomFog
# RandomGamma
# RandomGravel
# RandomRain
# RandomShadow
# RandomSnow
# RandomSunFlare	
# Solarize
# Spatter
# Superpixels
# TemplateTransform
# ToFloat
# ToGray
# ToRGB
# ToSepia
# UnsharpMask
# ZoomBlur

# Spatial transforms (changes labels):
# Affine
# BBoxSafeRandomCrop	
# CenterCrop
# CoarseDropout
# Crop
# CropAndPad 
# CropNonEmptyMaskIfExists
# ElasticTransform
# Flip
# GridDistortion	
# GridDropout 	
# HorizontalFlip
# Lambda
# LongestMaxSize 
# MaskDropout 	
# NoOp 
# OpticalDistortion	
# PadIfNeeded 
# Perspective 
# PiecewiseAffine 
# PixelDropout 	
# RandomCrop 	
# RandomCropFromBorders 
# RandomCropNearBBox 
# RandomGridShuffle 	
# RandomResizedCrop 	
# RandomRotate90 
# RandomScale 
# RandomSizedBBoxSafeCrop  	
# RandomSizedCrop 
# Resize 
# Rotate
# SafeRotate
# ShiftScaleRotate 
# SmallestMaxSize 
# Transpose
# VerticalFlip


def get_transform(proba= 0.8, proba2=0.5):
    return alb.Compose([
        alb.OneOf([
        alb.HorizontalFlip(),
        alb.VerticalFlip(),
        ], p=proba),

        
        alb.Rotate(p=proba2),
        alb.RandomBrightnessContrast(p=proba2),
        alb.RandomScale(p=proba2),
        alb.ShiftScaleRotate(p=proba2),
        alb.RandomGamma(p=proba2),

        alb.OneOf([
        alb.OpticalDistortion(),
        alb.ElasticTransform(),
        alb.GridDistortion()
        ], p=proba),


        alb.OneOf([
        alb.MotionBlur(),
        alb.MedianBlur(),
        alb.GaussianBlur(),
        alb.ZoomBlur(max_factor=1.125),
        alb.Blur(),
        ], p=proba),



        
        alb.RGBShift(p=proba2),
        alb.HueSaturationValue(p=proba2),
        #alb.Crop(),
        alb.GaussNoise(p=proba2),
        
    ])

def rotate_inner(image, groundtruth):


    rgood = False
    while not rgood:
        x = random.randint(0, image.shape[1] - 1)
        y = random.randint(0, image.shape[0] - 1)
        try:
            r = random.randint(40, min(x, y, image.shape[1] - x, image.shape[0] - y))
            rgood = True
        except:
            pass
            
    # Create a circular mask
    mask = np.zeros_like(image)
    cv2.circle(mask, (x, y), r, (255, 255, 255), thickness=-1)

    # Randomly generate angle theta in [0, 2*pi]
    theta = random.uniform(0, 2 * np.pi)

    # Rotate the circular area within Cx, Cy, r in image I with angle theta
    M = cv2.getRotationMatrix2D((x, y), np.degrees(theta), 1)
    rotated_area = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    rotated_area_gt = cv2.warpAffine(groundtruth, M, (groundtruth.shape[1], groundtruth.shape[0]))

    # Combine the rotated area with the original image
    I_augmented = cv2.bitwise_and(rotated_area, mask) + cv2.bitwise_and(image, cv2.bitwise_not(mask))
    G_augmented = cv2.bitwise_and(rotated_area_gt, mask[:, :, 0]) + cv2.bitwise_and(groundtruth, cv2.bitwise_not(mask[:, :, 0]))


    return I_augmented, G_augmented

In [35]:
images = [load_image(IMAGE_DIR + FILES[i]) for i in range(N_IMGS)]
masks = [load_image(GT_DIR + FILES[i]) for i in range(N_IMGS)]
print(len(images), len(masks))

100 100


In [36]:
save_augmented(images, masks, probas1=np.arange(0,9)/10, probas2=np.arange(0,9)/10, probas3=[0, 0, 0.7, 0, 0.1, 0.2, 0.3, 0.4, 0.7])